# Lập Trình GPT2 TỪ ĐẦU

## 1. Setup

In [1]:
! pip install --q torch tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.0 MB/s eta 0:00:00


In [2]:
! gdown 1-5UxJb_lGFf6iUXGi2-1yfrYy2WpApZT

Downloading...
From: https://drive.google.com/uc?id=1-5UxJb_lGFf6iUXGi2-1yfrYy2WpApZT
To: /content/chantuoi_docs.txt
100% 82.2M/82.2M [00:00<00:00, 124MB/s]


## 2. Chuẩn bị dữ liệu

In [3]:
with open('/content/chantuoi_docs.txt' , 'r', encoding='utf-8') as f:
  text = f.read().lower()
print(len(text))
text[:100]

62946178


'6 cách massage mắt giúp giảm mệt mỏi, trẻ hóa hiệu quả\nchia sẻ các cách massage mắt giúp giảm mệt mỏ'

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'the device is {device}')

class GPT2Dataset(Dataset):
  def __init__(self, text, tokenizer, block_size = 512, overlap_size = 128 ):
    self.text = text
    self.tokenizer = tokenizer
    self.block_size = block_size - 1 # trừ 1 vì thêm token BOS và EOS
    self.overlap_size = overlap_size
    self.device = device
    # lấy 2 id cuối cùng làm token_id bắt dầu và kết thúc
    self.BOS_token = 50249
    self.EOS_token = 50250
    # tokenize
    self.token_ids = tokenizer.encode(text)
    print(f'token_ids range {min(self.token_ids)} to {max(self.token_ids)}')
    # chia chunks dữ liệu có overlap
    self.input_ids , self.target_ids = self.create_chunks()
  def create_chunks(self):
    # input
    input_ids = []
    # nhãn
    target_ids = []
    for i in range(0 , len(self.token_ids), self.overlap_size):
      if i + self.block_size + 1 < len(self.token_ids):
        input_ids.append(
            torch.concat( (torch.tensor([self.BOS_token]), torch.tensor(self.token_ids[i:i+self.block_size]))).to(self.device)
        )
        target_ids.append(
            torch.concat( (torch.tensor(self.token_ids[i+1:i+self.block_size+1]), torch.tensor([self.EOS_token]))).to(self.device)
        )
    return input_ids , target_ids
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx] , self.target_ids[idx]

def train_test_split(dataset , train_ratio = 0.8):
  train_size = int(train_ratio * len(dataset))
  test_size = len(dataset) - train_size
  print(f'split dataset with train size : {train_size} | test size : {test_size}')
  train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
  return train_dataset , test_dataset

the device is cuda


In [5]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
dataset = GPT2Dataset(text , tokenizer)

token_ids range 0 to 50248


In [6]:
from torch.utils.data import DataLoader

train_dataset , test_dataset = train_test_split(dataset, 0.8)
train_loader = DataLoader(train_dataset , batch_size=16 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=16 , shuffle=False)

split dataset with train size : 341438 | test size : 85360


In [7]:
len(train_loader), len(test_loader)

(21340, 5335)

## 3. Xây dựng kiến trúc mô hình GPT2
* Embedding layer
* Positional Embedding layer
* Multi-head Attention
* Feed-Forward
* layer Normalization
* Transformer block
* GPT2

In [22]:
import torch
import torch.nn as nn
class PositionalEmbedding(nn.Module):
  def __init__(self, d_model, max_length = 512 ):
    super().__init__()
    self.d_model = d_model
    self.positional_emb = nn.Embedding(max_length, d_model)

  def forward(self, x ):
    seq_len = x.shape[1]
    positions = torch.arange(0, seq_len, device= x.device).unsqueeze(0)
    return x + self.positional_emb(positions)

class MultiHeadAttention(nn.Module):
  def __init__(self, n_heads , d_model):
    super().__init__()
    self.n_heads = n_heads
    self.d_model = d_model
    self.head_dim = d_model // n_heads
    self.fc = nn.Linear(self.n_heads * self.head_dim , self.d_model)
  def forward(self , Q, K, V, mask = None):
    # các kích thước
    batch_size = Q.shape[0]
    Q_seq_length = Q.shape[1]
    K_seq_length = K.shape[1]
    V_seq_length = V.shape[1]
    # chia Q, K, V thành n_head
    Q_heads = torch.reshape(Q, (batch_size, Q_seq_length, self.n_heads, self.head_dim)).permute(0, 2 , 1 , 3).to(device) #[N, n_head, Q_seq_length , head_dim]
    K_heads = torch.reshape(K, (batch_size, K_seq_length, self.n_heads, self.head_dim)).permute(0, 2 , 1 , 3).to(device) #[N, n_head, K_seq_length , head_dim]
    V_heads = torch.reshape(V, (batch_size, V_seq_length, self.n_heads, self.head_dim)).permute(0, 2 , 1 , 3).to(device) #[N, n_head, V_seq_length, head_dim]

    energy =torch.matmul(Q_heads , K_heads.permute(0, 1 , 3 , 2 )) / (self.d_model ** 0.5) # [N, n_head, Q_seq_length , K_seq_length]

    # áp dụng mask
    if mask is not None:
      energy = energy.masked_fill(mask == 0, -1e20)

    attention = torch.softmax(energy , dim = -1) # [N, n_head, Q_seq_length , K_seq_length]

    attention = torch.matmul(attention , V_heads) # [N , n_head, Q_seq_length, head_dim ]

    # đưa out trờ về kích thước giống với đầu vào ban đầu
    out = attention.permute(0 , 2 , 1 , 3 )
    out = out.reshape(batch_size , Q_seq_length , self.n_heads * self.head_dim )
    return self.fc(out) # [N , Q_seq_length, d_model]

class FeedForward(nn.Module):
  def __init__(self, d_model, expand_dim ):
    super().__init__()
    self.d_model = d_model
    self.expand_dim = expand_dim
    self.fc_block = nn.Sequential(
        nn.Linear(d_model , expand_dim),
        nn.GELU(),
        nn.Linear(expand_dim , d_model)
    )
  def forward(self , x):
    return self.fc_block(x)

class TransformersBlock(nn.Module):
  def __init__(self , d_model, expand_dim, n_heads, dropout_rate):
    super().__init__()
    self.d_model = d_model
    self.expand_dim = expand_dim
    self.n_heads = n_heads
    self.dropout_rate = dropout_rate
    # lần lượt các lớp của transformer
    self.attention = MultiHeadAttention(n_heads , d_model)
    self.layer_norm1 = nn.LayerNorm(d_model)
    self.feed_forward = FeedForward(d_model , expand_dim)
    self.layer_norm2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout_rate)
  def forward( self , Q, K , V , mask = None):
    """
      Q, K, V : [N , seq_len, d_model]
      mask : [N, 1 , seq_len, seq_len]

      return x : [N , seq_len, d_model]
    """
    x = self.attention(Q , K , V , mask)
    x = self.dropout(self.layer_norm1(x + Q))
    x = self.feed_forward(x)
    x = self.dropout(self.layer_norm2(x + Q))
    return x



In [23]:
class GPT2( nn.Module):
    def __init__(
        self,
        n_layers ,
        n_heads ,
        d_model ,
        expand_dim ,
        vocab_size ,
        max_length,
        dropout_rate
    ):
        super().__init__()
        self.n_layers = n_layers
        self.n_heads = n_heads
        self.d_model = d_model
        self.expand_dim = expand_dim
        self.vocab_size = vocab_size

        # các lớp của GPT2
        self.embedding = nn.Embedding(vocab_size , d_model)
        self.positional_embedding = PositionalEmbedding(d_model , max_length)
        self.transformer_blocks = nn.ModuleList([
            TransformersBlock(d_model , expand_dim , n_heads , dropout_rate) for _ in range(n_layers)
        ])
        self.fc = nn.Linear(d_model , vocab_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self , x , mask = None):
        """
          x : [N, seq_len]
          mask : [N, 1, seq_len, seq_len]
        """
        N, seq_len = x.shape[ 0 ], x.shape[ 1 ]
        x = self.embedding(x)
        x = self.positional_embedding(x)
        x = self.dropout(x)
        # cho qua các lớp transformer
        for block in self.transformer_blocks:
          x = block(x , x , x , mask)
        x = self.fc(x)
        return x

In [10]:
def create_mask(seq_len):
  return torch.tril(torch.ones(seq_len , seq_len)).unsqueeze(0).unsqueeze(0).to(device) # [1, 1 , seq_len, seq_len ]


In [24]:
import torch
import torch.nn as nn

model = GPT2(
    n_layers = 1,
    n_heads = 8,
    d_model = 512,
    expand_dim = 1024,
    vocab_size = tokenizer.n_vocab + 2,
    max_length = 512,
    dropout_rate = 0.1
).to(device)

## Thiết lập quá trình huấn luyện

In [25]:
optimizer = torch.optim.Adam(model.parameters() , lr = 3e-4)
criterion = nn.CrossEntropyLoss()

In [26]:
import math
import torch

def calculate_perplexity(loss):
    return math.exp(loss)

def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=3):
    best_val_perplexity = float('inf')
    best_model_path = 'best_model.pth'

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch_idx, batch in enumerate(train_loader):
            inputs, targets = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()

            # Create a causal mask for the current batch sequence length
            batch_size, seq_length = inputs.shape
            heads = model.n_heads

            mask = create_mask(seq_length).to(device)
            mask = mask.expand(batch_size, 1, seq_length, seq_length)

            # print(f"Mask shape: {mask.shape}")

            outputs = model(inputs, mask=mask)
            loss = criterion(outputs.transpose(1, 2), targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            batch_perplexity = calculate_perplexity(loss.item())
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], "
                  f"Train Loss: {loss.item():.4f}, Train Perplexity: {batch_perplexity:.4f}")

        # Validation loop with masking
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                inputs, targets = batch[0].to(device), batch[1].to(device)
                mask = create_mask(inputs.size(1)).to(device)
                outputs = model(inputs, mask=mask)
                loss = criterion(outputs.transpose(1, 2), targets)
                val_loss += loss.item()
                batch_perplexity = calculate_perplexity(loss.item())
                print(f"Validation, Batch [{batch_idx + 1}/{len(val_loader)}], "
                      f"Validation Loss: {loss.item():.4f}, Validation Perplexity: {batch_perplexity:.4f}")

        # Calculate average validation perplexity
        avg_val_loss = val_loss / len(val_loader)
        avg_val_perplexity = calculate_perplexity(avg_val_loss)
        print(f"Epoch [{epoch + 1}/{num_epochs}] Validation Summary: "
              f"Average Validation Loss: {avg_val_loss:.4f}, Average Validation Perplexity: {avg_val_perplexity:.4f}")

        # Check if this is the best model so far based on validation perplexity
        if avg_val_perplexity < best_val_perplexity:
            best_val_perplexity = avg_val_perplexity
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation perplexity: {best_val_perplexity:.4f}")

    print("Training complete. Best model saved at:", best_model_path)


In [ ]:
# Train the model and validate
train(model, train_loader, test_loader, optimizer, criterion, device)

Streaming output truncated to the last 5000 lines.
Epoch [1/3], Batch [6758/21340], Train Loss: 3.3089, Train Perplexity: 27.3560
Epoch [1/3], Batch [6759/21340], Train Loss: 3.2738, Train Perplexity: 26.4103
Epoch [1/3], Batch [6760/21340], Train Loss: 3.3226, Train Perplexity: 27.7323
Epoch [1/3], Batch [6761/21340], Train Loss: 3.2187, Train Perplexity: 24.9957
Epoch [1/3], Batch [6762/21340], Train Loss: 3.2357, Train Perplexity: 25.4241
Epoch [1/3], Batch [6763/21340], Train Loss: 3.2290, Train Perplexity: 25.2539
Epoch [1/3], Batch [6764/21340], Train Loss: 3.2792, Train Perplexity: 26.5536
Epoch [1/3], Batch [6765/21340], Train Loss: 3.1489, Train Perplexity: 23.3106
Epoch [1/3], Batch [6766/21340], Train Loss: 3.2783, Train Perplexity: 26.5308
Epoch [1/3], Batch [6767/21340], Train Loss: 3.2192, Train Perplexity: 25.0072
Epoch [1/3], Batch [6768/21340], Train Loss: 3.2610, Train Perplexity: 26.0752
Epoch [1/3], Batch [6769/21340], Train Loss: 3.3132, Train Perplexity: 27.4741
E

## Inference with Beam Search